# Tugas Klasifikasi dan Prediksi
Tugas: Memprediksi apakah setelah membeli customer akan mengembalikan barang (di Eropa ada perlindungan konsumen yang membuat konsumen dapat mengembalikan barang tanpa alasan asal dalam periode tertentu). Pengembalian barang yang berlebih dapat merugikan pemilik toko. Minimal akurasi yagn harus dicapai adalah 70%.


### Kelompok 3
- Ayesha Ali Firdaus (2101990)
- Ibnu Adeng Kurnia (2101769)
- Ihsan Ghozi Zulfikar (2103303)
- M. Fadhillah Nursyawal (2107135)
- M. Labib Husain (2101989)

In [461]:
# Import library
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import date
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
import pickle
import xgboost as xgb
import joblib

# Import Data Training
df = pd.read_csv("C:\\Users\\AYESHA ALI FIRDAUS\\DatMin\\orders_train.csv")

### ===================PRAPROSES===================

Membuat handling_time dan memperbaiki deliveryDate

In [462]:
# Dibuat null karena tanggal lahir tidak bisa di tebak
df.loc[df.dateOfBirth == "?", "dateOfBirth"] = np.nan
df.loc[(df.dateOfBirth == "1655-04-19", "dateOfBirth")] = np.nan

# Ingin mengubah deliveryDate yang anomali menjadi median dari waktu antar dikurangin waktu delivery yang tidak anomali
df.loc[df.deliveryDate == "?", "deliveryDate"] = np.nan #-------------- Menjadikan NULL terlebih dahulu data yang "?"
df.loc[df.deliveryDate < "2012", "deliveryDate"] = np.nan #------------ Mengubah menjadi NULL data yang kurang dari 2012
df['orderDate'] = pd.to_datetime(df['orderDate']) #-------------------- Ubah tipe data menjadi date agar bisa menghitung hari
df['deliveryDate'] = pd.to_datetime(df['deliveryDate']) #-------------- Ubah tipe data menjadi date agar bisa menghitung hari
df['handling_time'] = df['deliveryDate'] - df['orderDate'] #----------- Menambahkan atribut total hari pengantaran dari data 
                                                                        #yang tidak anomali {di dapat dari mengurangi tanggal 
                                                                        #pengiriman dengan tanggal order}
df.loc[df.handling_time < "0 days", "handling_time"] = np.nan #-------- Menjadikan NULL data yang anomali akibat deliveryDate 
                                                                        #yang anomali {contoh: 1990}
df['handling_time'].fillna(df['handling_time'].median(),inplace=True) # Mengisikan handling time yang NULL dengan median
df['deliveryDate'] = df['orderDate'] + df['handling_time'] #----------- Semua delivery date yang "?" dan kurang dari 2012
                                                                        #ditambahin median dari handling_time

# Mengubah handling time menjadi int
df['handling_time'] = df['handling_time'].dt.days #-------- Mengubah tipe data yang awalnya date menjadi float64 karena diambil 
                                                            #harinya saja
df["handling_time"] = df["handling_time"].astype("Int64") # mengubah tipe data menjadi int64

Menambahkan atribut umur

In [463]:
# Menambahkan atribut age ke dalam dataset
df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth']) #------------------- Mengubah tipe data dateOfBirth menjadi date
df['age'] = df['orderDate'] - df['dateOfBirth'] # Menghitung umur dari tahun 2013 akhir dikurangi dengan tanggal lahir
df['age'] = df['age'].dt.days #------------------------------------------- Mengubah tipe data date menjadi float64
df["age"] = df["age"].astype("Int64") #----------------------------------- Mengubah tipe data float64 menjadi int64
df["age"] = df["age"]  / 365 #-------------------------------------------- Mengubah hari menjadi tahun dengan dibagi oleh
                                                                           #365 hari
df["age"] = df["age"].astype("Int64") #----------------------------------- Mengubah tipe data float64 menjadi int64 {akibat 
                                                                           #dari hasil pembagian sebelumnya}
df['age'].fillna(df['age'].median(),inplace=True) #----------------------- Mengisikan age yang NULL dengan median dari age

# Drop atribut yang memiliki null dan anomali
df = df.drop(df.index[df.age >90])
df = df.drop(df.index[df.age <10])

Mengubah tipe data salutation

In [464]:
# Mengubah tipe data salutation menjadi category
df["salutation"] = df["salutation"].astype("category")

Membuat atribut umur akun

In [465]:
# Mengubah tipe data creationDate menjadi date
df['creationDate'] = pd.to_datetime(df['creationDate'])

# Menghitung umur akun, didapat dari waktu order dikurangin waktu pembuatan akun
df['AccountAge'] = df['orderDate'] - df['creationDate']
df['AccountAge'] = df['AccountAge'].dt.days

Mengubah atribut size menjadi ukuran internasional (XXS, XS, S, M, L, XL, XXL, XXXL)

In [466]:
# Mengubah size huruf kecil menjadi kapital
df['size'] = df['size'].replace({"xs":"XS", "s":"S", "m":"M", "l":"L", "xl":"XL", "xxl":"XXL", "xxxl":"XXXL"})
# Mengubah size menjadi angka
df['size'] = df['size'].replace({"XS":"36", "S":"38", "M":"40", "L":"42", "XL":"44", "XXL":"46", "XXXL":"48"})
# Menghilangkan tanda +
df['size'] = df['size'].replace({"9+":"9", "10+":"10", "8+":"8", "42+":"42", "4+":"4", "6+":"6", "7+":"7"
                                , "36+":"36", "39+":"39", "40+":"40", "5+":"5", "38+":"38", "11+":"11", "37+":"37"
                                , "41+":"41", "3+":"3", "44+":"44", "43+":"43", "13+":"13", "45+":"45", "46+":"46"
                                , "2+":"2", "12+":"12", "unsized":"-1"})

# Mengubah tipe data menjadi integer
df["size"] = df["size"].astype("int64")

# Menetapakn size berdasarkan range yang kami tetapkan
df.loc[df['size'].between(28, 31) , "size"] = -2
df.loc[df['size'].between(1, 3) | df['size'].between(32, 35) , "size"] = -3
df.loc[df['size'].between(4, 7) | df['size'].between(36, 39) , "size"] = -4
df.loc[df['size'].between(8, 11) | df['size'].between(40, 43) , "size"] = -5
df.loc[df['size'].between(12, 15) | df['size'].between(44, 47) , "size"] = -6
df.loc[df['size'].between(16, 19) | df['size'].between(48, 51) , "size"] = -7
df.loc[df['size'].between(20, 23) | df['size'].between(52, 55) , "size"] = -8
df.loc[df['size'].between(24, 27) | df['size'].between(56, 59) , "size"] = -9
df.loc[df['size'].between(60, 5000) , "size"] = -10

# Mengubah tipe data menjadi string
df["size"] = df["size"].astype("object")
# Mengembalikan size angka menjadi size internasional
df['size'] = df['size'].replace({-1:"unsized", -2:"XXS", -3:"XS", -4:"S"
                                 , -5:"M", -6:"L", -7:"XL", -8:"XXL", -9:"XXXL", -10:"XXXXL"})

# Menampilkan size setelah di praproses
df["size"].unique()

array(['M', 'S', 'L', 'unsized', 'XXXXL', 'XS', 'XXXL', 'XXL', 'XL',
       'XXS'], dtype=object)

Mengubah atribut color menjadi lebih sederhana

In [467]:
df.color = df.color.replace({"dark denim":"black", "dark navy":"blue", "ash":"grey",
                             "bordeaux":"red", "mahagoni":"red", "gold":"yellow",
                             "dark oliv":"green", "striped":"other", "anthracite":"black",
                             "antique pink":"red", "floral":"other", "baltic blue":"blue",
                             "nature":"other", "ancient":"other", "curry":"yellow",
                             "turquoise":"blue", "navy":"blue", "brown":"brown", "aubergine":"red",
                             "mocca":"brown", "blau":"blue", "basalt":"grey", "azure":"blue",
                             "coral":"red", "pallid":"grey", "petrol":"blue", "silver":"grey",
                             "habana":"red", "darkblue":"blue", "beige":"yellow", "mint":"green",
                             "khaki":"brown", "hibiscus":"red", "orange":"red", "yellow":"yellow",
                             "black":"black", "blue":"blue", "purple":"red", "almond":"brown", 
                             "red":"red", "berry":"red", "grey":"grey", "ocher":"brown",
                             "avocado":"green", "magenta":"red", "olive":"green", "white":"white",
                             "denim":"other", "pink":"red", "stained":"brown", "kanel":"brown",
                             "green":"green", "jade":"green", "aquamarine":"blue", "aqua":"blue",
                             "ecru":"grey", "iron":"grey", "fuchsia":"red", "ingwer":"red",
                             "cognac":"brown", "terracotta":"brown", "apricot":"yellow",
                             "graphite":"grey", "crimson":"red", "lemon":"yellow", "oliv":"green",
                             "leopard":"yellow", "amethyst":"red", "aviator":"other",
                             "bronze":"brown", "brwon":"brown", "caramel":"yellow",
                             "champagner":"yellow", "cobalt blue":"blue", "copper coin":"brown",
                             "cortina mocca":"brown", "creme":"yellow", "curled":"other",
                             "currant purple":"red", "dark garnet":"black", "dark grey":"grey",
                             "ebony":"black", "ivory":"white", "mango":"yellow", "opal":"other",
                             "perlmutt":"other", "vanille":"white"})

# Menampilkan color setelah di praproses
df["color"].unique()

array(['other', 'brown', 'yellow', 'green', 'black', 'red', 'blue',
       'grey', 'white', '?'], dtype=object)

Menambahkan atribut bulan

In [468]:
df['month'] = df['orderDate'].dt.month

Menambahkan atribut manufacture yang sering dikembalikan

In [529]:
# Membuat tabel dengan jumlah return dan no return
manufacturer_return = pd.DataFrame(pd.crosstab(df['manufacturerID'], df['returnShipment']))
manufacturer_return.columns = ['No_Return', 'Return']

# Menghitung jumlah total penjualan
manufacturer_return['Tot_Sales'] = manufacturer_return['No_Return'] + manufacturer_return['Return']

# Variabel sementara untuk menyimpan Tot_Sales
first_column = manufacturer_return.pop('Tot_Sales')

# Tot_Sales ke kolom pertama
manufacturer_return.insert(0, 'Tot_Sales', first_column)

# Menghitung persentase berdasarkan baris
manufacturer_return['No_Return_Pct'] = round(manufacturer_return['No_Return'] / manufacturer_return['Tot_Sales'], 2)
manufacturer_return['Return_Pct'] = round(manufacturer_return['Return'] / manufacturer_return['Tot_Sales'], 2)

# Mengurutkan berdasarkan persentase pengembalian
manufacturer_return = manufacturer_return.sort_values(by = 'Return_Pct', ascending = False)
manufacturer_return['manufacturerID'] = manufacturer_return.index
manufacturer_return = manufacturer_return.reset_index(drop = True)

# Memilih lebih dari 100 pesanan
manufacturer_return = manufacturer_return[manufacturer_return['Return_Pct'] > 0.55]
manufacturer_return.head()

,Tot_Sales,No_Return,Return,No_Return_Pct,Return_Pct,manufacturerID
0,2,0,2,0.00,1.00,93
1,2,0,2,0.00,1.00,143
2,3,0,3,0.00,1.00,105
3,140,23,117,0.16,0.84,96
4,185,34,151,0.18,0.82,101


In [470]:
df['manufacturer>55%'] = np.where(np.isin(df['manufacturerID'], manufacturer_return['manufacturerID'].values), 1, 0, )
df['manufacturer>55%'].value_counts()

0    394508
1     81111
Name: manufacturer>55%, dtype: int64

In [ ]:
Menambahkan atribut item yang sering dikembalikan

In [471]:
# Membuat tabel dengan jumlah return dan no return
item_return = pd.DataFrame(pd.crosstab(df['itemID'], df['returnShipment']))
item_return.columns = ['No_Return', 'Return']

# Menghitung jumlah total penjualan
item_return['Tot_Sales'] = item_return['No_Return'] + item_return['Return']

# Variabel sementara untuk menyimpan Tot_Sales
first_column = item_return.pop('Tot_Sales')
  
# Tot_Sales ke kolom pertama
item_return.insert(0, 'Tot_Sales', first_column)

# Menghitung persentase berdasarkan baris
item_return['No_Return_Pct'] = round(item_return['No_Return'] / item_return['Tot_Sales'], 2)
item_return['Return_Pct'] = round(item_return['Return'] / item_return['Tot_Sales'], 2)

# Mengurutkan berdasarkan persentase pengembalian
item_return = item_return.sort_values(by = 'Return_Pct', ascending = False)
item_return['itemID'] = item_return.index
item_return = item_return.reset_index(drop = True)

# Memilih lebih dari 100 pesanan
item_return = item_return[item_return['Return_Pct'] > 0.55]
item_return


,Tot_Sales,No_Return,Return,No_Return_Pct,Return_Pct,itemID
0,1,0,1,0.00,1.00,1082
1,2,0,2,0.00,1.00,1261
2,1,0,1,0.00,1.00,1362
3,1,0,1,0.00,1.00,3005
4,1,0,1,0.00,1.00,1259
...,...,...,...,...,...,...
988,281,124,157,0.44,0.56,259
989,75,33,42,0.44,0.56,2811
990,123,54,69,0.44,0.56,408
991,137,60,77,0.44,0.56,2462


In [473]:
df['item>55%'] = np.where(np.isin(df['itemID'], item_return['itemID'].values), 1, 0, )
df['item>55%'].value_counts()

0    317465
1    158154
Name: item>55%, dtype: int64

In [474]:
df['feature>55%'] = np.where(np.isin(df['itemID'], item_return['itemID'].values) 
                    & (np.isin(df['manufacturerID'], manufacturer_return['manufacturerID'].values)), 1, 0, )
df['feature>55%'].value_counts()

0    416314
1     59305
Name: feature>55%, dtype: int64

In [475]:
# Membuat rata" harga dari tiap item
avg_price_p_item = pd.DataFrame(df.groupby(by=['itemID'])['price'].mean())
avg_price_p_item['itemID'] = avg_price_p_item.index
avg_price_p_item = avg_price_p_item.reset_index(drop = True)
avg_price_p_item = avg_price_p_item.rename(columns = {'price': "Item_Average_Price"})
avg_price_p_item['Item_Average_Price'] = round(avg_price_p_item['Item_Average_Price'], 2)
avg_price_p_item.head()

,Item_Average_Price,itemID
0,111.80,1
1,30.50,2
2,23.07,3
3,90.14,4
4,52.52,5


In [476]:
# Memasukkan proses sebelumnya ke data frame
df = pd.merge(df, avg_price_p_item, on = 'itemID', how = 'left')
df

,orderItemID,orderDate,deliveryDate,itemID,size,color,manufacturerID,price,customerID,salutation,...,creationDate,returnShipment,handling_time,age,AccountAge,month,manufacturer>55%,item>55%,feature>55%,Item_Average_Price
0,1,2012-04-01,2012-04-03,186,M,other,25,69.90,794,Mrs,...,2011-04-25,0,2,47,342,4,1,1,1,69.47
1,2,2012-04-01,2012-04-03,71,M,brown,21,69.95,794,Mrs,...,2011-04-25,1,2,47,342,4,1,1,1,58.09
2,3,2012-04-01,2012-04-03,71,M,yellow,21,69.95,794,Mrs,...,2011-04-25,1,2,47,342,4,1,1,1,58.09
3,4,2012-04-02,2012-04-05,22,M,green,14,39.90,808,Mrs,...,2012-01-04,0,3,52,89,4,0,0,0,31.12
4,5,2012-04-02,2012-04-05,151,S,black,53,29.90,825,Mrs,...,2011-02-16,0,3,47,411,4,1,1,1,27.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475614,481088,2013-03-31,2013-04-09,2561,S,blue,1,139.90,7032,Mr,...,2011-03-12,1,9,50,750,3,0,1,0,139.90
475615,481089,2013-03-31,2013-04-02,2669,S,grey,18,130.00,86610,Mrs,...,2012-03-24,0,2,47,372,3,0,1,0,130.00
475616,481090,2013-03-31,2013-04-03,2458,L,black,5,99.90,86611,Mr,...,2013-03-31,0,3,39,0,3,0,1,0,99.90
475617,481091,2013-03-31,2013-04-03,2659,L,black,49,99.90,86611,Mr,...,2013-03-31,0,3,39,0,3,1,1,1,99.90


In [ ]:
Membuat fitur gabungan antara item handling_time

In [477]:
# interaction 1
df['interaction_1'] = np.where((df['item>55%'] <= 0.5) & 
                                     (df['handling_time'] > 10.5) & 
                                     (df['handling_time'] <= 11.5), 1, 0)

# interaction 2
df['interaction_2'] = np.where((df['item>55%'] <= 0.5) & 
                                     (df['handling_time'] <= 10.5) & 
                                     (df['price'] <= 29.425), 1, 0)

# interaction 3
df['interaction_3'] = np.where((df['item>55%'] > 0.5) & 
                                     (df['handling_time'] <= 10.5), 1, 0)

# interaction 4
df['interaction_4'] = np.where((df['item>55%'] > 0.5) & 
                                     (df['handling_time'] > 11.5), 1, 0)

In [478]:
df['price_higher_average'] = df['price']/df['Item_Average_Price']
df['price_higher_average'] = np.where(df['price_higher_average'] > 1.4, 1, 0)

X = df[df['price_higher_average'] == 1]
X.shape

(11914, 27)

Menambahkan atribut waktu penanganan yang sering dikembalikan

In [528]:
# Membuat tabel dengan jumlah return dan no return
timeDelivery_return = pd.DataFrame(pd.crosstab(df['handling_time'], df['returnShipment']))
timeDelivery_return.columns = ['No_Return', 'Return']

# Menghitung jumlah total penjualan
timeDelivery_return['Tot_Sales'] = timeDelivery_return['No_Return'] + timeDelivery_return['Return']

# Variabel sementara untuk menyimpan Tot_Sales
first_column = timeDelivery_return.pop('Tot_Sales')

# Tot_Sales ke kolom pertama
timeDelivery_return.insert(0, 'Tot_Sales', first_column)

# Menghitung persentase berdasarkan baris
timeDelivery_return['No_Return_Pct'] = round(timeDelivery_return['No_Return'] / timeDelivery_return['Tot_Sales'], 2)
timeDelivery_return['Return_Pct'] = round(timeDelivery_return['Return'] / timeDelivery_return['Tot_Sales'], 2)

# Mengurutkan berdasarkan persentase pengembalian
timeDelivery_return = timeDelivery_return.sort_values(by = 'Return_Pct', ascending = False)
timeDelivery_return['handling_time'] = timeDelivery_return.index
timeDelivery_return = timeDelivery_return.reset_index(drop = True)
## timeDelivery_return

# Memilih lebih dari 100 pesanan
timeDelivery_return = timeDelivery_return[timeDelivery_return['Return_Pct'] > 0.55]
timeDelivery_return.head()

,Tot_Sales,No_Return,Return,No_Return_Pct,Return_Pct,handling_time
0,1,0,1,0.0,1.0,149
1,1,0,1,0.0,1.0,165
2,2,0,2,0.0,1.0,150
3,1,0,1,0.0,1.0,151
4,1,0,1,0.0,1.0,158


In [481]:
df['timeDelivery>55%'] = np.where(np.isin(df['handling_time'], timeDelivery_return['handling_time'].values), 1, 0, )
df['timeDelivery>55%'].value_counts()

0    453599
1     22020
Name: timeDelivery>55%, dtype: int64

Menambahkan atribut customerID yang sering mengembalikan

In [482]:
# Membuat tabel dengan jumlah return dan no return
customer_return = pd.DataFrame(pd.crosstab(df['customerID'], df['returnShipment']))
customer_return.columns = ['No_Return', 'Return']

# Menghitung jumlah total penjualan
customer_return['Tot_Sales'] = customer_return['No_Return'] + customer_return['Return']

# Variabel sementara untuk menyimpan Tot_Sales
first_column = customer_return.pop('Tot_Sales')

# Tot_Sales ke kolom pertama
customer_return.insert(0, 'Tot_Sales', first_column)

# Menghitung persentase berdasarkan baris
customer_return['No_Return_Pct'] = round(customer_return['No_Return'] / customer_return['Tot_Sales'], 2)
customer_return['Return_Pct'] = round(customer_return['Return'] / customer_return['Tot_Sales'], 2)

# Mengurutkan berdasarkan persentase pengembalian
customer_return = customer_return.sort_values(by = 'Return_Pct', ascending = False)
customer_return['customerID'] = customer_return.index
customer_return = customer_return.reset_index(drop = True)

# Memilih lebih dari 100 pesanan
customer_return = customer_return[customer_return['Return_Pct'] > 0.55]
customer_return

df['customerID>55%'] = np.where(np.isin(df['customerID'], customer_return['customerID'].values), 1, 0, )

Menambahkan atribut warna yang sering dikembalikan

In [483]:
# Membuat tabel dengan jumlah return dan no return
color_return = pd.DataFrame(pd.crosstab(df['color'], df['returnShipment']))
color_return.columns = ['No_Return', 'Return']

# Menghitung jumlah total penjualan
color_return['Tot_Sales'] = color_return['No_Return'] + color_return['Return']

# Variabel sementara untuk menyimpan Tot_Sales
first_column = color_return.pop('Tot_Sales')

# Tot_Sales ke kolom pertama
color_return.insert(0, 'Tot_Sales', first_column)

# Menghitung persentase berdasarkan baris
color_return['No_Return_Pct'] = round(color_return['No_Return'] / color_return['Tot_Sales'], 2)
color_return['Return_Pct'] = round(color_return['Return'] / color_return['Tot_Sales'], 2)

# Mengurutkan berdasarkan persentase pengembalian
color_return = color_return.sort_values(by = 'Return_Pct', ascending = False)
color_return['color'] = color_return.index
color_return = color_return.reset_index(drop = True)

# Memilih lebih dari 100 pesanan
color_return = color_return[color_return['Return_Pct'] > 0.55]
color_return

,Tot_Sales,No_Return,Return,No_Return_Pct,Return_Pct,color
0,14519,6401,8118,0.44,0.56,other


In [484]:
df['color>55%'] = np.where(np.isin(df['color'], color_return['color'].values), 1, 0, )
df['color>55%'].value_counts()

0    461100
1     14519
Name: color>55%, dtype: int64

In [485]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475619 entries, 0 to 475618
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   orderItemID           475619 non-null  int64         
 1   orderDate             475619 non-null  datetime64[ns]
 2   deliveryDate          475619 non-null  datetime64[ns]
 3   itemID                475619 non-null  int64         
 4   size                  475619 non-null  object        
 5   color                 475619 non-null  object        
 6   manufacturerID        475619 non-null  int64         
 7   price                 475619 non-null  float64       
 8   customerID            475619 non-null  int64         
 9   salutation            475619 non-null  category      
 10  dateOfBirth           426727 non-null  datetime64[ns]
 11  state                 475619 non-null  object        
 12  creationDate          475619 non-null  datetime64[ns]
 13 

### ===================KLASIFIKASI===================

In [509]:
# Pembuangan atribut yang tidak terpakai
df_temp = df
df_temp = df_temp.drop(['orderItemID', 'dateOfBirth','creationDate', 'orderDate', 'deliveryDate'],axis=1)

In [511]:
# Membuat One-Hot Encoding
df_temp = pd.get_dummies(data=df_temp, columns = ['state', 'size', 'color', 'salutation', 'month'])

In [512]:
# Menentukan Kelas Target
le = preprocessing.LabelEncoder()
le.fit(df_temp.returnShipment)
Y = le.transform(df_temp.returnShipment)
X = df_temp.drop("returnShipment",axis=1)

In [513]:
# Memisahkan antara data train dengan data test
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=123)

with open("C:\\Users\\AYESHA ALI FIRDAUS\\DatMin\\orders_class.pickle", 'wb') as fp:
    pickle.dump(X_train.columns, fp)

Prediksi menggunakan Naive Bayes

In [518]:
# Naive Bayes
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc_NB = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc_NB))
print(classification_report(Y_test, Y_pred))

Akurasi 0.6241852739582019
              precision    recall  f1-score   support

           0       0.61      0.74      0.67     49240
           1       0.64      0.50      0.56     45884

    accuracy                           0.62     95124
   macro avg       0.63      0.62      0.62     95124
weighted avg       0.63      0.62      0.62     95124



Prediksi menggunakan Decision Tree

In [519]:
# Decision Tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc_DT = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc_DT))
print(classification_report(Y_test, Y_pred))

Akurasi 0.636148606030024
              precision    recall  f1-score   support

           0       0.65      0.65      0.65     49240
           1       0.62      0.62      0.62     45884

    accuracy                           0.64     95124
   macro avg       0.64      0.64      0.64     95124
weighted avg       0.64      0.64      0.64     95124



Prediksi menggunakan Random Forest

In [520]:
#random Forest
clf = RandomForestClassifier(n_estimators=150, random_state=123)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc_RF = accuracy_score(Y_test, Y_pred)
print("Akurasi {}".format(acc_RF))
print(classification_report(Y_test, Y_pred))

Akurasi 0.7114187796980783
              precision    recall  f1-score   support

           0       0.71      0.74      0.73     49240
           1       0.71      0.68      0.70     45884

    accuracy                           0.71     95124
   macro avg       0.71      0.71      0.71     95124
weighted avg       0.71      0.71      0.71     95124



Prediksi menggunakan XGBoost

In [521]:
# Menjalankan XGBoost
clf = xgb.XGBClassifier(objective = "binary:logistic", eval_metric = "mlogloss", 
                        max_depth = 20, gamma=0.8, subsample = 0.9,learning_rate=0.09, 
                        n_estimators = 150, seed = 27,
                        colsample_bytree = 0.9, nthread=4)

clf.fit(X_train, Y_train)# PCA
Y_pred = clf.predict(X_test)
acc_XGB = accuracy_score(Y_test, Y_pred)

# Mencetak hasil dari XGBoost
print("Akurasi {}".format(acc_XGB))
print(classification_report(Y_test, Y_pred))

Akurasi 0.7150035742819898
              precision    recall  f1-score   support

           0       0.72      0.74      0.73     49240
           1       0.71      0.69      0.70     45884

    accuracy                           0.72     95124
   macro avg       0.71      0.71      0.71     95124
weighted avg       0.71      0.72      0.71     95124



In [523]:
df_acc = pd.DataFrame({'Naive_Bayes': [acc_NB], 'Decission_Tree': [acc_DT],
                       'Random_Forest': [acc_RF], 'XGBoost': [acc_XGB]})
df_acc

,Naive_Bayes,Decission_Tree,Random_Forest,XGBoost
0,0.624185,0.636149,0.711419,0.715004


In [524]:
# Untuk mengethaui atribut yang berperan dalam melakukan prediksi
feature_importances = pd.DataFrame(clf.feature_importances_,index = 
                        X_train.columns, columns = ['importance']).sort_values('importance',ascending=False)

print(feature_importances)

                         importance
customerID>55%             0.561487
item>55%                   0.077222
interaction_3              0.020614
interaction_4              0.008089
size_unsized               0.007641
...                             ...
color_red                  0.004253
interaction_2              0.003906
salutation_Company         0.003681
color_?                    0.003518
salutation_not reported    0.003115

[72 rows x 1 columns]


In [530]:
# Melihat atribut yang tidak begitu berpengaruh
feature_importances.iloc[50:72]

,importance
size_XS,0.004547
color_green,0.004540
month_2,0.004535
size_S,0.004526
state_Bavaria,0.004522
color_white,0.004509
age,0.004506
state_Baden-Wuerttemberg,0.004499
size_L,0.004484
month_9,0.004480
